In [32]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input, Reshape
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv1D, Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

In [2]:
randomDim = 100

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)

In [3]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [4]:
noise = Input(shape=(randomDim,))

x = Dense(256, kernel_initializer=initializers.RandomNormal(stddev=0.02))(noise)
x = LeakyReLU(0.2)(x)
x = Dense(512)(x)
x = LeakyReLU(0.2)(x)
x = Dense(1024)(x)
x = LeakyReLU(0.2)(x)
x = Dense(784, activation='tanh')(x)
gen_out = Reshape((28, 28, 1))(x)

generator = Model(noise, gen_out)
generator.compile(loss='binary_crossentropy', optimizer=adam)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
__________

In [40]:
discriminator = Sequential()
discriminator.add(Dense(784, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(PrimaryCap((28, 28), dim_vector=8, n_channels=24, kernel_size=9, strides=2, padding='valid'))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
discriminator.summary()

ValueError: Layer primarycap_conv2d was called with an input that isn't a symbolic tensor. Received type: <class 'tuple'>. Full input: [(28, 28)]. All inputs to the layer should be tensors.

In [22]:
# Combined network
discriminator.trainable = True
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [23]:
dLosses = []
gLosses = []

In [24]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/gan_loss_epoch_%d.png' % epoch)

In [25]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/gan_generated_image_epoch_%d.png' % epoch)

In [26]:
# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator.save('models/gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('models/gan_discriminator_epoch_%d.h5' % epoch)

In [27]:
def train(epochs=1, batchSize=128):
    batchCount = int(X_train.shape[0] / batchSize)
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batchCount)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)].reshape(batchSize, 28, 28, 1)

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            # print np.shape(imageBatch), np.shape(generatedImages)
            print(imageBatch.shape, generatedImages.shape)
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = True
            print('Noise:', noise.shape)
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)
        
        # Plot the progress
        print('Epoch: {0}, Dloss: {1}, Dacc: {2}%, Gloss: {3}'.format(e, dloss[0], dloss[1]*100, gloss))

        if e == 1 or e % 10 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)

In [28]:
train(30, 64)

Epochs: 30
Batch size: 64
Batches per epoch: 937
--------------- Epoch 1 ---------------


  0%|                                                  | 0/937 [00:00<?, ?it/s]

(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)


  0%|                                          | 1/937 [00:03<59:26,  3.81s/it]

(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)
(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)


...

(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)
(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)
(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)


 37%|██████████████▊                         | 348/937 [00:18<00:31, 18.85it/s]

(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)
(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)
(64, 28, 28, 1) (64, 28, 28, 1)
Noise: (64, 100)


KeyboardInterrupt: 

 37%|██████████████▊                         | 348/937 [00:30<00:50, 11.60it/s]